# デンドログラムの活用

このノートブックでは、デンドログラムの作成と解釈を学習します。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)

# データの生成
X, y_true = make_blobs(n_samples=50, centers=3, random_state=42)
X_scaled = StandardScaler().fit_transform(X)

# 異なる結合方法でのデンドログラム
methods = ['single', 'complete', 'average', 'ward']
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

for i, method in enumerate(methods):
    row, col = i // 2, i % 2
    
    if method == 'ward':
        # Ward法はユークリッド距離のみ
        linkage_matrix = linkage(X_scaled, method=method)
    else:
        # その他の方法は距離行列を使用
        dist_matrix = pdist(X_scaled)
        linkage_matrix = linkage(dist_matrix, method=method)
    
    # デンドログラムの作成
    dendrogram(linkage_matrix, ax=axes[row, col], truncate_mode='level', p=5)
    axes[row, col].set_title(f'Dendrogram ({method.capitalize()})')
    axes[row, col].set_xlabel('Sample Index')
    axes[row, col].set_ylabel('Distance')

plt.tight_layout()
plt.show()

# 最適なクラスタ数の決定
def find_optimal_clusters(linkage_matrix, max_clusters=10):
    distances = linkage_matrix[:, 2]
    diff = np.diff(distances, 2)
    optimal_clusters = np.argmax(diff) + 2
    return min(optimal_clusters, max_clusters)

# Ward法での最適クラスタ数
ward_linkage = linkage(X_scaled, method='ward')
optimal_k = find_optimal_clusters(ward_linkage)

print(f"=== デンドログラムの解釈 ===")
print(f"最適なクラスタ数: {optimal_k}")
print(f"Ward法の結合距離: {ward_linkage[:, 2]}")

# クラスタの可視化
from sklearn.cluster import AgglomerativeClustering

# 異なるクラスタ数での結果
cluster_counts = [2, 3, 4, 5]
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

for i, k in enumerate(cluster_counts):
    row, col = i // 2, i % 2
    
    # 階層クラスタリング
    clustering = AgglomerativeClustering(n_clusters=k)
    labels = clustering.fit_predict(X_scaled)
    
    # 可視化
    axes[row, col].scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels, cmap='viridis', alpha=0.7)
    axes[row, col].set_title(f'Clusters: {k}')
    axes[row, col].set_xlabel('Feature 1')
    axes[row, col].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

print(f"\n=== クラスタリング結果 ===")
for k in cluster_counts:
    clustering = AgglomerativeClustering(n_clusters=k)
    labels = clustering.fit_predict(X_scaled)
    print(f"クラスタ数 {k}: ラベル分布 {np.bincount(labels)}")
